In [6]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go 

from openpyxl import Workbook
from IPython.display import display

pd.set_option('display.max_columns', 40) # выводить больше столбцов
pd.set_option('display.max_rows', 40)   # выводить больше строк
plt.style.use('default')    # корректный вывод графиков в темной теме 

# fig.show(renderer='notebook') #включение анимации

#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests



r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')


# https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip

OSError: [Errno 9] Bad file descriptor

In [ ]:

diab = pd.read_csv('data/diabetes_data.csv')
diab.info()
with pd.ExcelWriter('task_befor_85.xlsx') as writer:  
    diab.to_excel(writer, sheet_name='Start_csv')


In [ ]:
# fig = px.histogram(diab, x="Pregnancies",  marginal="box", nbins=30, width=1000 , height= 500, color='Outcome')
# fig.show()

In [ ]:
diab_81 = diab.drop_duplicates()
diab_81.shape
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_81.to_excel(writer, sheet_name='drop_dup_81')


In [ ]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diab_81.columns:
    #наибольшая относительная частота в признаке
    top_freq = diab_81[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diab_81[col].nunique() / diab_81[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
print(low_information_cols)

In [ ]:
diab_82 = diab_81.drop(low_information_cols, axis=1)
diab_82.head()
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_82.to_excel(writer, sheet_name='drop_gender_82')


In [ ]:
# Задание 8.3
# Попробуйте найти пропуски в данных с помощью метода insull().
# Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI
# обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
# Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

diab_83 = diab_82.copy()
col_to_change = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in col_to_change:
    diab_83[col] = diab_83[col].replace({0: np.nan})
display(diab_83.head())

round(diab_83['Insulin'].isna().value_counts(normalize=True)[True], 2)

In [ ]:
# Задание 8.4
# Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?
display(diab_83.shape)
n = diab_83.shape[0] #число строк в таблице
thresh_col = n*0.7
diab_84 = diab_83.dropna(how='any', thresh=thresh_col, axis=1)
display(diab_84)
display(diab_84.shape)
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_84.to_excel(writer, sheet_name='drop_Insulin_84')

In [ ]:
# Задание 8.5
# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

display(diab_84.shape)

m = diab_84.shape[1]
tresh_line = m - 2 # убираем те записи где больее 2х Nan
print(f'triesh_line = {tresh_line}')
diab_85 = diab_84.dropna(how='any', thresh=tresh_line, axis=0)

with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_85.to_excel(writer, sheet_name='drop_lines_85')
display(diab_85.info())

display(diab_85.shape)

# values = {
#     'one': df['one'].mean(),
#     'two': df['two'].mean(),
#     'three': df['three'].mean(),
#     'four': df['four'].mode()[0]
# }


In [ ]:
# Задание 8.6
# # В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
print(round(diab_85['SkinThickness'].mean(),1))
diab_86= diab_85.copy()
cols = list(diab_86.columns)
cols.remove('Outcome')
print(col_to_change)
values = {
    'Glucose': diab_86['Glucose'].median(),
    'BloodPressure': diab_86['BloodPressure'].median(),
    'SkinThickness': diab_86['SkinThickness'].median(),
    'BMI': diab_86['BMI'].median()}
#заполняем оставшиеся записи константами в соответствии со словарем values
diab_86 = diab_86.fillna(values)
display(round(diab_86['SkinThickness'].mean(),1))

In [ ]:
# Задание 8.7

# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr_mod(data, feature, left =1.5, right= 1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diab_86, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
def outliers_z_score_mod(data, feature, left = 3, right = 3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(diab_86, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


In [ ]:
outliers, cleaned = outliers_iqr_mod(diab_86, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers, cleaned = outliers_iqr_mod(diab_86, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).



In [ ]:
Data_Parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
Data_Company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')
display(Data_Company)
display(Data_Parsing)

In [ ]:
import re
signs_of_signs = ['~','_','-', '/', '\\\\']
Data_Parsing.replace(to_replace=signs_of_signs, value="", regex=True, inplace=True)
Data_Parsing

In [ ]:
Data_merged = Data_Parsing.merge(Data_Company, how='left', left_on=['producer_id',  'producer_color'], right_on=['item_id', 'color_id'])
Data_merged['Diff_%'] = round((Data_merged['price'] - Data_merged['current price']) / Data_merged['current price']*100 , 2)
Data_merged

In [ ]:
fig = px.box(Data_merged, x = 'brand_x', y='Diff_%', color='Category_x')
fig.show()

In [ ]:
Data_merged['brand_x'].value_counts().index

In [ ]:
new_merged = pd.DataFrame()
all_outliers = pd.DataFrame()
for brand in Data_merged['brand_x'].value_counts().index:
    for category in Data_merged['Category_x'].value_counts().index:
        mask1 = Data_merged['brand_x'] == brand
        mask2 =Data_merged['Category_x'] == category
        mask = mask1 & mask2
        outliers, cleaned = outliers_iqr_mod(Data_merged[mask], 'Diff_%')
        print(f'Число выбросов по методу Тьюки:{brand}-{category} = {outliers.shape[0]}')
        print(f'Результирующее число записей:{brand}-{category} =  {cleaned.shape[0]}')
        new_merged = pd.concat([new_merged, cleaned], axis= 0)
        all_outliers = pd.concat([all_outliers, outliers], axis= 0)
fig = px.box(new_merged, x = 'brand_x', y='Diff_%', color='Category_x')
fig.show()


In [ ]:
Data_merged.groupby(['brand_x', 'Category_x'], as_index=False)['Diff_%'].agg(['mean', 'std'])

In [ ]:


# hh_df1 = pd.read_csv('data/hh_database.csv', sep= ';')

# test_hh = pd.read_csv('data/hh_database.csv', sep= ';', nrows=1000)

# hh_df['Образование и ВУЗ'] = hh_df['Образование и ВУЗ'].apply(lambda x : x.split(sep=' ')[0] +' '+ x.split(sep=' ')[1]).astype('category')
# hh_df['Образование и ВУЗ'].value_counts()


# hh_df.head()
# sex_age = pd.DataFrame()
# sex_age['Пол, возраст'] = hh_df['Пол, возраст'].apply(lambda x : x.split(sep=' ')[0])
# sex_age.head()